# Piece Detector Training — YOLOv8s (12 classes) on Google Colab T4

**What this notebook does:**
1. Mounts your Google Drive (upload `chess_pieces/` there first).
2. Installs `ultralytics`.
3. Rewrites `data.yaml` with absolute Colab paths and the correct 12 class names.
4. Trains a YOLOv8s piece-detection model for 150 epochs at 640×640.
5. Prints per-class and overall validation metrics.
6. Downloads `best.pt` → save it as **`piece_detector.pt`** and drop it into `python-ml-service/models/`.

---
### Before you run
* Make sure **Runtime → Change runtime type → GPU (T4)** is selected.
* Upload the entire `chess_pieces/` folder to **My Drive** root (i.e. `/content/drive/MyDrive/chess_pieces/`).

### Class mapping (must match your YOLO label files exactly)
| ID | Class |
|---|---|
| 0 | black_bishop |
| 1 | black_king |
| 2 | black_knight |
| 3 | black_pawn |
| 4 | black_queen |
| 5 | black_rook |
| 6 | white_bishop |
| 7 | white_king |
| 8 | white_knight |
| 9 | white_pawn |
| 10 | white_queen |
| 11 | white_rook |

---
## Step 1 — Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
PIECE_DATASET_ROOT = '/content/drive/MyDrive/chess_pieces'
assert os.path.isdir(PIECE_DATASET_ROOT), (
    f"Dataset folder not found at {PIECE_DATASET_ROOT}. "
    "Upload chess_pieces/ to the root of My Drive."
)
print('Drive mounted. Dataset root confirmed.')

---
## Step 2 — Install ultralytics

In [ ]:
!pip install ultralytics -q

---
## Step 3 — Rewrite data.yaml with absolute Colab paths + 12 classes

In [ ]:
import yaml

DATA_YAML_PATH = os.path.join(PIECE_DATASET_ROOT, 'data.yaml')

# Read existing yaml (keeps roboflow metadata block intact)
with open(DATA_YAML_PATH) as f:
    data = yaml.safe_load(f)

# Overwrite paths to absolute Colab locations
data['train'] = os.path.join(PIECE_DATASET_ROOT, 'train', 'images')
data['val']   = os.path.join(PIECE_DATASET_ROOT, 'valid', 'images')
data['test']  = os.path.join(PIECE_DATASET_ROOT, 'test',  'images')

# Ensure 12-class config is locked in (matches the label files exactly)
data['nc'] = 12
data['names'] = [
    'black_bishop',   # 0
    'black_king',     # 1
    'black_knight',   # 2
    'black_pawn',     # 3
    'black_queen',    # 4
    'black_rook',     # 5
    'white_bishop',   # 6
    'white_king',     # 7
    'white_knight',   # 8
    'white_pawn',     # 9
    'white_queen',    # 10
    'white_rook',     # 11
]

with open(DATA_YAML_PATH, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

# Print the final yaml for visual verification
with open(DATA_YAML_PATH) as f:
    print(f.read())

---
## Step 4 — Verify GPU + dataset integrity

In [ ]:
import torch
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))

# File counts per split
for split in ('train', 'valid', 'test'):
    img_dir = os.path.join(PIECE_DATASET_ROOT, split, 'images')
    lbl_dir = os.path.join(PIECE_DATASET_ROOT, split, 'labels')
    imgs = len([f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))])
    lbls = len([f for f in os.listdir(lbl_dir) if f.endswith('.txt')])
    print(f'{split:>6s} — images: {imgs}, labels: {lbls}')

# Class-ID distribution across training labels
from collections import Counter
counter = Counter()
lbl_dir = os.path.join(PIECE_DATASET_ROOT, 'train', 'labels')
for fname in os.listdir(lbl_dir):
    if not fname.endswith('.txt'):
        continue
    with open(os.path.join(lbl_dir, fname)) as f:
        for line in f:
            cls_id = int(line.strip().split()[0])
            counter[cls_id] += 1

CLASS_NAMES = data['names']
print('\n--- Training label distribution ---')
for cls_id in sorted(counter.keys()):
    print(f'  {cls_id:>2d}  {CLASS_NAMES[cls_id]:<16s}  {counter[cls_id]:>5d} instances')

---
## Step 5 — Train YOLOv8s

| Hyper-parameter | Value | Why |
|---|---|---|
| model | yolov8s.pt | Small — fast on T4, good enough for 12 classes |
| epochs | 150 | More epochs than board model because 12 classes need more convergence |
| imgsz | 640 | Matches the Roboflow export resolution |
| batch | 16 | Safe for T4 16 GB |
| device | 0 | First GPU |
| patience | 30 | Early-stop window (wider than board because 12 classes) |
| lr0 | 0.01 | Default — works well for transfer from COCO |
| lrf | 0.01 | Final LR = lr0 × lrf |

> If mAP50 plateaus below 0.80 after 80 epochs, consider switching to `yolov8m.pt` (medium).
> That will roughly double the training time on T4 but adds ~10 % mAP ceiling.

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')          # pretrained COCO weights

results = model.train(
    data     = DATA_YAML_PATH,
    epochs   = 150,
    imgsz    = 640,
    batch    = 16,
    device   = 0,
    patience = 30,                   # early stopping
    lr0      = 0.01,
    lrf      = 0.01,
    project  = '/content/runs',
    name     = 'piece_detection',
    verbose  = True,
)

---
## Step 6 — Overall validation metrics

In [ ]:
metrics = model.metrics
print('\n===== Piece Detector — Overall Validation Metrics =====')
print(f"  mAP50      : {metrics.box.map50:.4f}")
print(f"  mAP50-95   : {metrics.box.map:.4f}")
print(f"  Precision  : {metrics.box.mp:.4f}")
print(f"  Recall     : {metrics.box.mr:.4f}")
print('=======================================================')

---
## Step 7 — Per-class validation metrics

In [ ]:
# ultralytics stores per-class AP in metrics.box.ap (list, one entry per class)
ap_list = metrics.box.ap50  # AP@IoU=0.5 per class

print('\n--- Per-Class AP@0.5 ---')
print(f'{"Class":<18s} {"AP50":>8s}')
print('-' * 28)
for i, name in enumerate(CLASS_NAMES):
    val = ap_list[i] if i < len(ap_list) else float('nan')
    flag = '  ⚠ low' if val < 0.70 else ''
    print(f'{name:<18s} {val:>7.3f}{flag}')
print('-' * 28)
print(f'{"Mean":<18s} {sum(ap_list)/len(ap_list):>7.3f}')

---
## Step 8 — Validate on the held-out test set

In [ ]:
test_results = model.val(
    data   = DATA_YAML_PATH,
    split  = 'test',
    imgsz  = 640,
    device = 0,
)
print('\n===== Test-Set Metrics =====')
print(f"  mAP50      : {test_results.box.map50:.4f}")
print(f"  mAP50-95   : {test_results.box.map:.4f}")
print(f"  Precision  : {test_results.box.mp:.4f}")
print(f"  Recall     : {test_results.box.mr:.4f}")
print('============================')

---
## Step 9 — Visual sample predictions on test images

In [ ]:
from IPython.display import display, Image as IPImage
import glob

test_imgs = sorted(glob.glob(os.path.join(PIECE_DATASET_ROOT, 'test', 'images', '*.*')))
sample    = test_imgs[:6]          # 6 sample images

pred_results = model.predict(
    source = sample,
    save   = True,
    show   = False,
    device = 0,
    conf   = 0.3,                   # lower conf for visual review
)

pred_dir = sorted(glob.glob('/content/runs/predict*'))[-1]
for img_path in sorted(glob.glob(os.path.join(pred_dir, '*.*'))):
    display(IPImage(filename=img_path))

---
## Step 10 — Confusion matrix

In [ ]:
# ultralytics saves a confusion_matrix.png in the val output folder
import glob
from IPython.display import display, Image as IPImage

cm_files = sorted(glob.glob('/content/runs/piece_detection/val*/confusion_matrix.png'))
if cm_files:
    display(IPImage(filename=cm_files[-1]))
else:
    print('Confusion matrix image not found — check /content/runs/piece_detection/val*/')

---
## Step 11 — Download best.pt

This downloads **`best.pt`** to your browser.
Save it as **`piece_detector.pt`** and place it in:
`python-ml-service/models/piece_detector.pt`

In [ ]:
from google.colab import files

BEST_PT = '/content/runs/piece_detection/weights/best.pt'
assert os.path.isfile(BEST_PT), (
    f'best.pt not found at {BEST_PT}. '
    'Check /content/runs/ manually.'
)

files.download(BEST_PT)
print('Download started. Save the file as  piece_detector.pt')

---
### Done
Place the downloaded file at:
```
ThesisBookProject/
  └── python-ml-service/
        └── models/
              └── piece_detector.pt   ← here
```

### Troubleshooting
| Symptom | Fix |
|---|---|
| mAP50 < 0.75 after 150 epochs | Switch to `yolov8m.pt` in Step 5 and re-run |
| One class dominates errors | Check label count (Step 4 printout); consider class weights or oversampling |
| OOM on T4 | Reduce `batch` to 8 |
| "RuntimeError: CUDA" | Restart runtime, re-run from Step 1 |